In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
import os
import glob
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import tree

In [2]:
os.chdir("D:/Data")
csv_data_path = 'D:/Data/'

In [10]:
traindata = pd.read_csv('D:/Data/preprocessed_train_DT_2018.csv')
testdata = pd.read_csv('D:/Data/preprocessed_test_DT_2018.csv')
traindata.pop("Unnamed: 0")
testdata.pop("Unnamed: 0")
traindata = traindata[20000:50000]
testdata = testdata[0:20000]
traindata.head()
testdata['Label'].value_counts()


1    11383
0     8617
Name: Label, dtype: int64

In [11]:
Y = traindata.pop('Label')
X = traindata.iloc[:,0:82]
C = testdata.pop('Label')
T = testdata.iloc[:,0:82]

X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 20000 to 49999
Data columns (total 80 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dst Port           30000 non-null  int64  
 1   Flow Duration      30000 non-null  int64  
 2   Tot Fwd Pkts       30000 non-null  int64  
 3   Tot Bwd Pkts       30000 non-null  int64  
 4   TotLen Fwd Pkts    30000 non-null  int64  
 5   TotLen Bwd Pkts    30000 non-null  int64  
 6   Fwd Pkt Len Max    30000 non-null  int64  
 7   Fwd Pkt Len Min    30000 non-null  int64  
 8   Fwd Pkt Len Mean   30000 non-null  float64
 9   Fwd Pkt Len Std    30000 non-null  float64
 10  Bwd Pkt Len Max    30000 non-null  int64  
 11  Bwd Pkt Len Min    30000 non-null  int64  
 12  Bwd Pkt Len Mean   30000 non-null  float64
 13  Bwd Pkt Len Std    30000 non-null  float64
 14  Flow Byts/s        30000 non-null  float64
 15  Flow Pkts/s        30000 non-null  float64
 16  Flow IAT Mean     

In [12]:
_traindata = np.array(X)
_trainlabel = np.array(Y)
_traindata = X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
_testdata = np.array(T)
_testlabel = np.array(C)
_testdata = T.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
_trainlabel

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [13]:
_testdata.head()

,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol_0,Protocol_6,Protocol_17
0,80,475048,5,3,220,472,220,0,44.0,98.386991,...,0.0,0,0,0.0,0.0,0,0,0,1,0
1,80,477314,5,3,215,467,215,0,43.0,96.150923,...,0.0,0,0,0.0,0.0,0,0,0,1,0
2,80,476789,5,3,215,467,215,0,43.0,96.150923,...,0.0,0,0,0.0,0.0,0,0,0,1,0
3,80,530813,5,3,215,467,215,0,43.0,96.150923,...,0.0,0,0,0.0,0.0,0,0,0,1,0
4,80,477602,5,3,215,467,215,0,43.0,96.150923,...,0.0,0,0,0.0,0.0,0,0,0,1,0


In [84]:
from genetic_selection import GeneticSelectionCV

In [87]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clfSVM = make_pipeline(StandardScaler(), SVC())
clfSVM.fit(_traindata, _trainlabel)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [88]:
from sklearn.metrics import confusion_matrix, zero_one_loss, accuracy_score
predSVM = clfSVM.predict(_testdata)
results = confusion_matrix(_testlabel, predSVM)
error = zero_one_loss(_testlabel, predSVM)
accuracy_score = accuracy_score(_testlabel, predSVM)
print("Decision Tree Classifier Results\n======================\nConfusion Matrix:\n{}\nError Value:{}\nAccuracy_Score:{}\n".format(results, error, accuracy_score))

Decision Tree Classifier Results
Confusion Matrix:
[[8571   46]
 [6071 5312]]
Error Value:0.30584999999999996
Accuracy_Score:0.69415



In [89]:
from joblib import dump, load
dump(clfSVM, 'svm_intrusion_detection_binary_classification.pkl')

['svm_intrusion_detection_binary_classification.pkl']

In [ ]:
from genetic_selection import GeneticSelectionCV
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


cv = GeneticSelectionCV(estimator=SVC(),
                                  cv=10,
                                  verbose=1,
                                  scoring="accuracy",
                                  max_features=20,
                                  n_population=50,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=40,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)

print(cv.fit(_traindata, _trainlabel))

Selecting features with genetic algorithm.
gen	nevals	avg                  	std                    	min          	max              
0  	50    	[ 0.664318 10.58    ]	[0.06050299 6.7619228 ]	[0.488 1.   ]	[ 0.7466 20.    ]
1  	26    	[-399.33038267   14.7       ]	[1959.72847964    5.0645829 ]	[-1.e+04  1.e+00]	[ 0.7466 23.    ]
2  	28    	[-799.34664867   16.18      ]	[2713.12465618    4.6376287 ]	[-1.e+04  6.e+00]	[ 0.7466 25.    ]
3  	34    	[-799.34960867   14.4       ]	[2713.12378343    5.26117858]	[-1.e+04  5.e+00]	[ 0.7466 25.    ]
4  	34    	[-799.344466   13.78    ]    	[2713.12529988    5.02509701]	[-1.e+04  6.e+00]	[ 0.7466 24.    ]
5  	33    	[-599.332798   11.46    ]    	[2375.03698352    4.80503902]	[-1.e+04  6.e+00]	[ 0.7466 22.    ]
6  	28    	[0.72193267 8.2       ]      	[0.05239292 3.57211422]      	[0.464 6.   ]    	[ 0.7466 20.    ]
7  	32    	[0.730664 6.82    ]          	[0.03858384 1.99689759]      	[0.58123333 6.        ]	[ 0.7466 15.    ]
8  	32    	[0.72751533 7

In [ ]:
from joblib import dump, load
dump(cv, 'svm_genetic_intrusion_detection_binary_classification.pkl')

In [10]:
cv = pd.read_pickle("svm_genetic_intrusion_detection_binary_classification.pkl")

UnpicklingError: invalid load key, '\x00'.

In [27]:
print(cv.support_)

[ True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False False False False False False
 False False False False False False False False]


In [89]:
from sklearn.metrics import confusion_matrix, zero_one_loss, accuracy_score,classification_report


pred = cv.predict(_testdata)
results = confusion_matrix(_testlabel, pred)
error = zero_one_loss(_testlabel, pred)
accuracy_score = accuracy_score(_testlabel, pred)
print("Decision Tree Classifier Results\n======================\nConfusion Matrix:\n{}\nError Value:{}\nAccuracy_Score:{}\n".format(results, error, accuracy_score))


Decision Tree Classifier Results
Confusion Matrix:
[[8545   72]
 [9683 1700]]
Error Value:0.48775
Accuracy_Score:0.51225



In [39]:
from sklearn.metrics import classification_report
print(classification_report(_testlabel, pred))

              precision    recall  f1-score   support

           0       0.47      0.99      0.64      8617
           1       0.96      0.15      0.26     11383

    accuracy                           0.51     20000
   macro avg       0.71      0.57      0.45     20000
weighted avg       0.75      0.51      0.42     20000



In [61]:
test.head()

,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol_0,Protocol_6,Protocol_17
30007,80,100012084,15,0,480,0,32,32,32.000000,0.000000,...,3461459.099,9009658,1011873,9.327821e+06,4.578369e+06,20000000,5983291,0,0,1
30009,443,118904661,12,11,602,3192,202,0,50.166667,71.514567,...,381315.817,782383,243121,5.880000e+07,1.947867e+04,58800000,58800000,0,1,0
30012,80,515972,5,3,214,466,214,0,42.800000,95.703709,...,0.000,0,0,0.000000e+00,0.000000e+00,0,0,0,1,0
30019,52899,36,1,1,0,0,0,0,0.000000,0.000000,...,0.000,0,0,0.000000e+00,0.000000e+00,0,0,0,1,0
30020,500,89479529,6,0,3000,0,500,500,500.000000,0.000000,...,0.000,4000318,4000318,2.136980e+07,1.528109e+07,41990303,7200355,0,0,1
